# Complete Analysis Pipeline

This notebook demonstrates a complete end-to-end analysis workflow, integrating all framework capabilities. Learn how to:

- **Run complete analysis pipeline** from data loading to reporting
- **Integrate all analysis modules** (core, preprocessing, experimental, quality)
- **Generate comprehensive reports** with visualizations
- **Export results** in multiple formats
- **Create reproducibility packages** for sharing and validation

## 🎯 Learning Objectives

By the end of this notebook, you will be able to:
1. Execute a complete analysis workflow from start to finish
2. Integrate multiple analysis types in a single pipeline
3. Generate comprehensive reports and visualizations
4. Export results for external tools (DragonFly, etc.)
5. Create reproducibility packages

## ⚠️ Prerequisites

- **All previous notebooks**: Understanding of individual analysis modules
- **Required packages**: Same as previous notebooks
- **XCT data**: Raw or segmented volume data

## 📖 Usage

1. Run all cells to initialize the widgets
2. Configure the complete pipeline
3. Select analysis modules to include
4. Run the complete pipeline
5. Review comprehensive results
6. Export reports and reproducibility packages


## 1. Setup and Imports


In [1]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys
import warnings
import json
from typing import Dict, List, Optional, Tuple, Any
from datetime import datetime

warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Check for ipywidgets
try:
    import ipywidgets as widgets
    from ipywidgets import HBox, VBox, Output, Tab, interactive
    from IPython.display import display, clear_output, HTML
    WIDGETS_AVAILABLE = True
except ImportError:
    WIDGETS_AVAILABLE = False
    print("❌ ipywidgets not available!")
    print("   Install with: pip install ipywidgets")

# Find project root
current_dir = Path().resolve()
if current_dir.name == 'notebooks':
    project_root = current_dir.parent
elif (current_dir / 'src').exists():
    project_root = current_dir
else:
    project_root = current_dir

# Add to path
sys.path.insert(0, str(project_root))
sys.path.insert(0, str(project_root / 'src'))

print("📦 Complete Analysis Pipeline")
print(f"   Project root: {project_root}")
print(f"   Widgets available: {WIDGETS_AVAILABLE}")


📦 Complete Analysis Pipeline
   Project root: /mnt/c/Users/kanha/Independent_Research/pbf-lbm-nosql-data-warehouse/XCT_Thermomagnetic_Analysis
   Widgets available: True


## 2. Load Framework Modules


In [ ]:
# Load all framework modules
try:
    from src.analyzer import XCTAnalyzer
    from src.core.metrics import compute_all_metrics
    from src.core.porosity import analyze_porosity_distribution
    from src.core.filament_analysis import estimate_filament_diameter
    from src.preprocessing.preprocessing import apply_filters
    from src.experimental.flow_analysis import comprehensive_flow_analysis
    from src.experimental.thermal_analysis import compute_thermal_resistance
    from src.experimental.energy_conversion import estimate_power_output
    from src.quality.dimensional_accuracy import comprehensive_dimensional_analysis
    from src.quality.uncertainty_analysis import comprehensive_uncertainty_analysis
    from src.quality.reproducibility import export_reproducibility_package
    from src.integration.dragonfly_integration import comprehensive_dragonfly_integration
    from src.core.visualization import create_analysis_report
    from src.utils.utils import load_volume, normalize_path
    
    print("✅ All modules loaded successfully")
except ImportError as e:
    print(f"❌ Error loading modules: {e}")
    import traceback
    traceback.print_exc()
    raise


✅ All modules loaded successfully


## 3. Interactive Complete Pipeline Dashboard

Use the interactive widgets below to configure and run a complete analysis pipeline.


In [3]:
if not WIDGETS_AVAILABLE:
    print("❌ Cannot create widgets - ipywidgets not available")
else:
    print("🎨 Creating interactive widgets...")
    
    # Initialize state
    analyzer = None
    pipeline_results = {}
    
    # ============================================
    # Section 1: Data Loading
    # ============================================
    
    file_path_text = widgets.Text(
        value='',
        placeholder='Enter file path to XCT volume',
        description='File Path:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='500px')
    )
    
    file_format_dropdown = widgets.Dropdown(
        options=['Auto-detect', 'DICOM', 'TIFF', 'RAW', 'NIfTI', 'NumPy'],
        value='Auto-detect',
        description='Format:',
        style={'description_width': 'initial'}
    )
    
    voxel_size_x = widgets.FloatText(value=0.1, description='Voxel X (mm):', style={'description_width': 'initial'})
    voxel_size_y = widgets.FloatText(value=0.1, description='Voxel Y (mm):', style={'description_width': 'initial'})
    voxel_size_z = widgets.FloatText(value=0.1, description='Voxel Z (mm):', style={'description_width': 'initial'})
    
    sample_name_text = widgets.Text(
        value='sample',
        description='Sample Name:',
        style={'description_width': 'initial'}
    )
    
    load_button = widgets.Button(
        description='📂 Load Volume',
        button_style='primary',
        layout=widgets.Layout(width='150px', height='40px')
    )
    
    volume_info_display = widgets.HTML(
        value="<p><i>No volume loaded</i></p>",
        layout=widgets.Layout(height='100px', overflow='auto')
    )
    
    # ============================================
    # Section 2: Pipeline Configuration
    # ============================================
    
    widgets.HTML("<h3>📋 Analysis Modules</h3>")
    
    enable_segmentation = widgets.Checkbox(value=True, description='Segmentation', indent=False)
    enable_preprocessing = widgets.Checkbox(value=True, description='Preprocessing', indent=False)
    enable_core_analysis = widgets.Checkbox(value=True, description='Core Analysis', indent=False)
    enable_porosity = widgets.Checkbox(value=True, description='Porosity Analysis', indent=False)
    enable_filament = widgets.Checkbox(value=True, description='Filament Analysis', indent=False)
    enable_flow = widgets.Checkbox(value=False, description='Flow Analysis', indent=False)
    enable_thermal = widgets.Checkbox(value=False, description='Thermal Analysis', indent=False)
    enable_energy = widgets.Checkbox(value=False, description='Energy Analysis', indent=False)
    enable_dimensional = widgets.Checkbox(value=False, description='Dimensional Accuracy', indent=False)
    enable_uncertainty = widgets.Checkbox(value=True, description='Uncertainty Analysis', indent=False)
    
    # Segmentation parameters
    segmentation_method = widgets.Dropdown(
        options=['otsu', 'adaptive', 'manual'],
        value='otsu',
        description='Segmentation:',
        style={'description_width': 'initial'}
    )
    
    # Preprocessing parameters
    filter_volume_check = widgets.Checkbox(value=False, description='Filter by Volume', indent=False)
    min_volume = widgets.FloatText(value=0.0, description='Min Volume (mm³):', style={'description_width': 'initial'}, disabled=True)
    
    # Flow parameters
    flow_direction = widgets.Dropdown(
        options=['X', 'Y', 'Z'],
        value='Z',
        description='Flow Direction:',
        style={'description_width': 'initial'},
        disabled=True
    )
    
    # Thermal parameters
    thermal_conductivity = widgets.FloatText(
        value=50.0,
        description='Thermal Conductivity (W/m·K):',
        style={'description_width': 'initial'},
        disabled=True
    )
    
    # Energy parameters
    temperature_gradient = widgets.FloatText(
        value=50.0,
        description='Temperature Gradient (K):',
        style={'description_width': 'initial'},
        disabled=True
    )
    
    # Dimensional parameters
    design_dim_x = widgets.FloatText(value=10.0, description='Design X (mm):', style={'description_width': 'initial'}, disabled=True)
    design_dim_y = widgets.FloatText(value=10.0, description='Design Y (mm):', style={'description_width': 'initial'}, disabled=True)
    design_dim_z = widgets.FloatText(value=10.0, description='Design Z (mm):', style={'description_width': 'initial'}, disabled=True)
    tolerance = widgets.FloatText(value=0.1, description='Tolerance (mm):', style={'description_width': 'initial'}, disabled=True)
    
    # ============================================
    # Section 3: Run Pipeline
    # ============================================
    
    run_pipeline_button = widgets.Button(
        description='🚀 Run Complete Pipeline',
        button_style='success',
        layout=widgets.Layout(width='250px', height='50px')
    )
    
    pipeline_results_display = widgets.HTML(
        value="<p><i>No pipeline results</i></p>",
        layout=widgets.Layout(height='300px', overflow='auto')
    )
    
    pipeline_visualization = Output(layout=widgets.Layout(height='500px'))
    
    # ============================================
    # Section 4: Export and Reporting
    # ============================================
    
    output_directory = widgets.Text(
        value='output',
        placeholder='Output directory path',
        description='Output Dir:',
        style={'description_width': 'initial'}
    )
    
    export_format = widgets.Dropdown(
        options=['JSON', 'CSV', 'Excel', 'All'],
        value='All',
        description='Export Format:',
        style={'description_width': 'initial'}
    )
    
    generate_report_check = widgets.Checkbox(value=True, description='Generate HTML Report', indent=False)
    export_dragonfly_check = widgets.Checkbox(value=False, description='Export to DragonFly', indent=False)
    create_reproducibility_check = widgets.Checkbox(value=True, description='Create Reproducibility Package', indent=False)
    
    export_button = widgets.Button(
        description='💾 Export Results',
        button_style='info',
        layout=widgets.Layout(width='200px')
    )
    
    export_status_display = widgets.HTML(
        value="<p><i>No exports</i></p>",
        layout=widgets.Layout(height='150px', overflow='auto')
    )
    
    # ============================================
    # Progress and Status
    # ============================================
    
    progress_bar = widgets.IntProgress(
        value=0,
        min=0,
        max=100,
        description='Progress:',
        style={'bar_color': '#2ecc71'},
        layout=widgets.Layout(width='400px')
    )
    
    status_display = widgets.HTML(
        value="<p>Ready</p>",
        layout=widgets.Layout(height='60px', overflow='auto')
    )
    
    print("✅ Widgets created successfully!")


🎨 Creating interactive widgets...
✅ Widgets created successfully!


In [ ]:
if WIDGETS_AVAILABLE:
    
    def load_volume_callback(button):
        """Load volume"""
        global analyzer
        
        file_path = file_path_text.value.strip()
        if not file_path:
            status_display.value = "<p style='color: red;'>Please enter a file path</p>"
            return
        
        # Normalize path for cross-platform compatibility (Windows, Linux, macOS, WSL)
        file_path_obj = normalize_path(file_path, base_path=project_root / 'data')
        if not file_path_obj.exists():
            # Try relative to project root
            file_path_obj = normalize_path(file_path, base_path=project_root)
            if not file_path_obj.exists():
                status_display.value = f"<p style='color: red;'>File not found: {file_path}</p>"
                return
        
        status_display.value = "<p>Loading volume...</p>"
        progress_bar.value = 20
        
        try:
            voxel_size = (float(voxel_size_x.value), float(voxel_size_y.value), float(voxel_size_z.value))
            analyzer = XCTAnalyzer(voxel_size=voxel_size, target_unit='mm')
            progress_bar.value = 40
            
            analyzer.load_volume(str(file_path_obj), normalize=True)
            progress_bar.value = 80
            
            info_html = f"""
            <h4>Volume Information</h4>
            <p><b>Shape:</b> {analyzer.volume.shape}</p>
            <p><b>Voxel Size:</b> {voxel_size} mm</p>
            <p><b>Volume Size:</b> {analyzer.volume.nbytes / (1024**2):.2f} MB</p>
            """
            volume_info_display.value = info_html
            
            progress_bar.value = 100
            status_display.value = "<p style='color: green;'>✅ Volume loaded successfully!</p>"
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error loading volume: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    def run_pipeline_callback(button):
        """Run complete analysis pipeline"""
        global analyzer, pipeline_results
        
        if analyzer is None or analyzer.volume is None:
            status_display.value = "<p style='color: red;'>Please load a volume first</p>"
            return
        
        status_display.value = "<p>Running complete pipeline...</p>"
        progress_bar.value = 0
        
        try:
            voxel_size = analyzer.voxel_size
            sample_name = sample_name_text.value.strip() or 'sample'
            results = {}
            
            # Step 1: Segmentation
            if enable_segmentation.value:
                progress_bar.value = 5
                status_display.value = "<p>Step 1/10: Segmenting volume...</p>"
                analyzer.segment(method=segmentation_method.value, refine=True)
                results['segmentation'] = {'method': segmentation_method.value, 'complete': True}
            
            # Step 2: Preprocessing
            if enable_preprocessing.value and analyzer.segmented_volume is not None:
                progress_bar.value = 15
                status_display.value = "<p>Step 2/10: Preprocessing...</p>"
                filters = {}
                if filter_volume_check.value:
                    filters['min_volume'] = float(min_volume.value)
                if filters:
                    filtered_volume, filter_stats = apply_filters(
                        analyzer.segmented_volume, voxel_size, filters
                    )
                    analyzer.segmented_volume = filtered_volume
                    results['preprocessing'] = filter_stats
            
            # Step 3: Core Analysis
            if enable_core_analysis.value and analyzer.segmented_volume is not None:
                progress_bar.value = 25
                status_display.value = "<p>Step 3/10: Computing core metrics...</p>"
                metrics = analyzer.compute_metrics()
                results['core_metrics'] = metrics
            
            # Step 4: Porosity Analysis
            if enable_porosity.value and analyzer.segmented_volume is not None:
                progress_bar.value = 35
                status_display.value = "<p>Step 4/10: Analyzing porosity...</p>"
                porosity_results = analyze_porosity_distribution(
                    analyzer.segmented_volume, voxel_size, printing_direction='z'
                )
                results['porosity'] = {k: v for k, v in porosity_results.items() if k != 'local_porosity_map'}
            
            # Step 5: Filament Analysis
            if enable_filament.value and analyzer.segmented_volume is not None:
                progress_bar.value = 45
                status_display.value = "<p>Step 5/10: Analyzing filaments...</p>"
                filament_results = estimate_filament_diameter(
                    analyzer.segmented_volume, voxel_size, direction='z'
                )
                results['filament'] = filament_results
            
            # Step 6: Flow Analysis
            if enable_flow.value and analyzer.segmented_volume is not None:
                progress_bar.value = 55
                status_display.value = "<p>Step 6/10: Analyzing flow...</p>"
                flow_results = comprehensive_flow_analysis(
                    analyzer.segmented_volume, flow_direction=flow_direction.value.lower(), voxel_size=voxel_size
                )
                results['flow'] = {k: {kk: vv for kk, vv in v.items() if not isinstance(vv, np.ndarray)} 
                                  for k, v in flow_results.items() if isinstance(v, dict)}
            
            # Step 7: Thermal Analysis
            if enable_thermal.value and analyzer.segmented_volume is not None:
                progress_bar.value = 65
                status_display.value = "<p>Step 7/10: Analyzing thermal...</p>"
                material_properties = {'thermal_conductivity': float(thermal_conductivity.value)}
                thermal_results = compute_thermal_resistance(
                    analyzer.segmented_volume, voxel_size, material_properties
                )
                results['thermal'] = thermal_results
            
            # Step 8: Energy Analysis
            if enable_energy.value and analyzer.segmented_volume is not None:
                progress_bar.value = 75
                status_display.value = "<p>Step 8/10: Analyzing energy conversion...</p>"
                energy_results = estimate_power_output(
                    analyzer.segmented_volume, voxel_size,
                    temperature_gradient=float(temperature_gradient.value)
                )
                results['energy'] = energy_results
            
            # Step 9: Dimensional Accuracy
            if enable_dimensional.value and analyzer.segmented_volume is not None:
                progress_bar.value = 85
                status_display.value = "<p>Step 9/10: Analyzing dimensional accuracy...</p>"
                design_specs = {
                    'dimensions': {
                        'x': float(design_dim_x.value),
                        'y': float(design_dim_y.value),
                        'z': float(design_dim_z.value)
                    },
                    'tolerance': float(tolerance.value)
                }
                dimensional_results = comprehensive_dimensional_analysis(
                    analyzer.segmented_volume, voxel_size, design_specs=design_specs
                )
                results['dimensional'] = {k: v for k, v in dimensional_results.items() 
                                        if not isinstance(v, (np.ndarray, dict)) or k == 'summary'}
            
            # Step 10: Uncertainty Analysis
            if enable_uncertainty.value and analyzer.segmented_volume is not None:
                progress_bar.value = 90
                status_display.value = "<p>Step 10/10: Analyzing uncertainty...</p>"
                if 'core_metrics' in results:
                    uncertainty_results = comprehensive_uncertainty_analysis(
                        analyzer.segmented_volume, voxel_size, metrics=results['core_metrics']
                    )
                    results['uncertainty'] = {k: v for k, v in uncertainty_results.items() 
                                              if not isinstance(v, (np.ndarray, dict)) or k in ['confidence_intervals', 'summary']}
            
            pipeline_results = results
            progress_bar.value = 100
            
            # Display results summary
            html = f"""
            <h4>🚀 Complete Pipeline Results</h4>
            <p><b>Sample:</b> {sample_name}</p>
            <p><b>Analysis Steps Completed:</b> {len(results)}</p>
            <h5>Results Summary:</h5>
            <ul>
            """
            for step_name, step_results in results.items():
                if isinstance(step_results, dict):
                    html += f"<li><b>{step_name.replace('_', ' ').title()}:</b> ✅ Complete</li>"
                else:
                    html += f"<li><b>{step_name.replace('_', ' ').title()}:</b> {step_results}</li>"
            html += "</ul>"
            
            # Add key metrics if available
            if 'core_metrics' in results:
                html += "<h5>Key Metrics:</h5><ul>"
                key_metrics = ['volume', 'surface_area', 'void_fraction', 'relative_density']
                for metric in key_metrics:
                    if metric in results['core_metrics']:
                        value = results['core_metrics'][metric]
                        html += f"<li><b>{metric.replace('_', ' ').title()}:</b> {value:.4f}</li>"
                html += "</ul>"
            
            pipeline_results_display.value = html
            
            # Visualize
            with pipeline_visualization:
                clear_output()
                fig, axes = plt.subplots(2, 2, figsize=(14, 10))
                
                # Metrics summary
                if 'core_metrics' in results:
                    metrics = results['core_metrics']
                    key_metrics = ['volume', 'surface_area', 'void_fraction', 'relative_density']
                    values = [metrics.get(m, 0) for m in key_metrics]
                    axes[0, 0].bar([m.replace('_', ' ').title() for m in key_metrics], values, alpha=0.7)
                    axes[0, 0].set_ylabel('Value', fontsize=11)
                    axes[0, 0].set_title('Core Metrics Summary', fontsize=12, fontweight='bold')
                    axes[0, 0].tick_params(axis='x', rotation=45)
                    axes[0, 0].grid(True, alpha=0.3, axis='y')
                
                # Analysis steps completed
                step_names = list(results.keys())
                axes[0, 1].barh(step_names, [1] * len(step_names), alpha=0.7, color='green')
                axes[0, 1].set_xlabel('Status', fontsize=11)
                axes[0, 1].set_title('Analysis Steps Completed', fontsize=12, fontweight='bold')
                axes[0, 1].set_xlim([0, 1.2])
                
                # Porosity if available
                if 'porosity' in results:
                    porosity_data = results['porosity']
                    if 'porosity_profile' in porosity_data:
                        profile = porosity_data['porosity_profile']
                        if isinstance(profile, dict) and 'porosity' in profile:
                            axes[1, 0].plot(profile['porosity'], linewidth=2)
                            axes[1, 0].set_xlabel('Position', fontsize=11)
                            axes[1, 0].set_ylabel('Porosity', fontsize=11)
                            axes[1, 0].set_title('Porosity Profile', fontsize=12, fontweight='bold')
                            axes[1, 0].grid(True, alpha=0.3)
                
                # Energy if available
                if 'energy' in results:
                    energy_data = results['energy']
                    power_output = energy_data.get('power_output', 0)
                    efficiency = energy_data.get('power_output', 0) / energy_data.get('heat_transfer_rate', 1) * 100 if energy_data.get('heat_transfer_rate', 0) > 0 else 0
                    axes[1, 1].bar(['Power Output\n(mW)', 'Efficiency\n(%)'], 
                                 [power_output * 1000, efficiency], alpha=0.7, color=['steelblue', 'coral'])
                    axes[1, 1].set_ylabel('Value', fontsize=11)
                    axes[1, 1].set_title('Energy Conversion', fontsize=12, fontweight='bold')
                    axes[1, 1].grid(True, alpha=0.3, axis='y')
                
                plt.tight_layout()
                plt.show()
            
            status_display.value = "<p style='color: green;'>✅ Complete pipeline finished successfully!</p>"
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error in pipeline: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    def export_callback(button):
        """Export results and generate reports"""
        global analyzer, pipeline_results
        
        if not pipeline_results:
            status_display.value = "<p style='color: red;'>Please run pipeline first</p>"
            return
        
        output_dir = output_directory.value.strip() or 'output'
        output_path = Path(output_dir)
        if not output_path.is_absolute():
            output_path = project_root / output_dir
        
        status_display.value = "<p>Exporting results...</p>"
        progress_bar.value = 10
        
        try:
            output_path.mkdir(parents=True, exist_ok=True)
            sample_name = sample_name_text.value.strip() or 'sample'
            exported_files = []
            
            # Export results in selected format
            export_fmt = export_format.value.lower()
            progress_bar.value = 20
            
            if export_fmt in ['json', 'all']:
                json_path = output_path / f'{sample_name}_results.json'
                with open(json_path, 'w') as f:
                    json.dump(pipeline_results, f, indent=2, default=str)
                exported_files.append(str(json_path))
            
            if export_fmt in ['csv', 'all']:
                # Flatten results for CSV
                flat_results = {}
                def flatten_dict(d, parent_key='', sep='_'):
                    items = []
                    for k, v in d.items():
                        new_key = f"{parent_key}{sep}{k}" if parent_key else k
                        if isinstance(v, dict):
                            items.extend(flatten_dict(v, new_key, sep=sep).items())
                        else:
                            items.append((new_key, v))
                    return dict(items)
                
                flat_data = flatten_dict(pipeline_results)
                df = pd.DataFrame([flat_data])
                csv_path = output_path / f'{sample_name}_results.csv'
                df.to_csv(csv_path, index=False)
                exported_files.append(str(csv_path))
            
            if export_fmt in ['excel', 'all']:
                # Create Excel with multiple sheets
                excel_path = output_path / f'{sample_name}_results.xlsx'
                with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
                    # Summary sheet
                    summary_data = {'Metric': [], 'Value': []}
                    if 'core_metrics' in pipeline_results:
                        for k, v in pipeline_results['core_metrics'].items():
                            if isinstance(v, (int, float)):
                                summary_data['Metric'].append(k)
                                summary_data['Value'].append(v)
                    pd.DataFrame(summary_data).to_excel(writer, sheet_name='Summary', index=False)
                    
                    # Individual sheets for each analysis type
                    for analysis_type, analysis_data in pipeline_results.items():
                        if isinstance(analysis_data, dict):
                            flat_data = {}
                            def extract_values(d, prefix=''):
                                for k, v in d.items():
                                    if isinstance(v, (int, float, str, bool)):
                                        flat_data[f"{prefix}{k}"] = v
                                    elif isinstance(v, dict):
                                        extract_values(v, prefix=f"{k}_")
                            extract_values(analysis_data)
                            if flat_data:
                                pd.DataFrame([flat_data]).to_excel(writer, sheet_name=analysis_type, index=False)
                exported_files.append(str(excel_path))
            
            progress_bar.value = 50
            
            # Generate HTML report
            if generate_report_check.value and analyzer is not None:
                progress_bar.value = 60
                try:
                    analyzer.generate_report(str(output_path), sample_name)
                    exported_files.append(str(output_path / f'{sample_name}_report.html'))
                except Exception as e:
                    status_display.value = f"<p style='color: orange;'>Warning: Could not generate HTML report: {e}</p>"
            
            # Export to DragonFly
            if export_dragonfly_check.value and analyzer is not None and analyzer.segmented_volume is not None:
                progress_bar.value = 70
                try:
                    dragonfly_dir = output_path / 'dragonfly_export'
                    dragonfly_results = comprehensive_dragonfly_integration(
                        analyzer.segmented_volume, analyzer.voxel_size,
                        analysis_results=pipeline_results,
                        output_dir=dragonfly_dir,
                        sample_name=sample_name
                    )
                    exported_files.append(str(dragonfly_dir))
                except Exception as e:
                    status_display.value = f"<p style='color: orange;'>Warning: Could not export to DragonFly: {e}</p>"
            
            # Create reproducibility package
            if create_reproducibility_check.value:
                progress_bar.value = 80
                try:
                    repro_dir = output_path / 'reproducibility_package'
                    from src.quality.reproducibility import AnalysisConfig, ProvenanceTracker, SeedManager
                    
                    # Create config
                    config = AnalysisConfig()
                    config.set('voxel_size', analyzer.voxel_size if analyzer else (0.1, 0.1, 0.1))
                    config.set('segmentation_method', segmentation_method.value)
                    
                    # Create provenance tracker
                    provenance = ProvenanceTracker()
                    provenance.add_step('load_volume', {'file_path': file_path_text.value})
                    provenance.add_step('pipeline_execution', {'steps': list(pipeline_results.keys())})
                    
                    # Export package
                    export_reproducibility_package(
                        repro_dir,
                        config=config,
                        provenance=provenance,
                        results=pipeline_results,
                        data_files=[file_path_text.value] if file_path_text.value else None
                    )
                    exported_files.append(str(repro_dir))
                except Exception as e:
                    status_display.value = f"<p style='color: orange;'>Warning: Could not create reproducibility package: {e}</p>"
            
            progress_bar.value = 100
            
            # Display export status
            html = f"""
            <h4>💾 Export Complete</h4>
            <p><b>Output Directory:</b> {output_path}</p>
            <p><b>Files Exported:</b> {len(exported_files)}</p>
            <h5>Exported Files:</h5>
            <ul>
            """
            for file_path in exported_files:
                html += f"<li>{Path(file_path).name}</li>"
            html += "</ul>"
            export_status_display.value = html
            
            status_display.value = f"<p style='color: green;'>✅ Export complete! Files saved to {output_path}</p>"
            
        except Exception as e:
            status_display.value = f"<p style='color: red;'>Error exporting: {e}</p>"
            progress_bar.value = 0
            import traceback
            traceback.print_exc()
    
    # Attach callbacks
    load_button.on_click(load_volume_callback)
    run_pipeline_button.on_click(run_pipeline_callback)
    export_button.on_click(export_callback)
    
    # Enable/disable widgets based on checkboxes
    def update_widgets(change):
        min_volume.disabled = not filter_volume_check.value
        flow_direction.disabled = not enable_flow.value
        thermal_conductivity.disabled = not enable_thermal.value
        temperature_gradient.disabled = not enable_energy.value
        design_dim_x.disabled = not enable_dimensional.value
        design_dim_y.disabled = not enable_dimensional.value
        design_dim_z.disabled = not enable_dimensional.value
        tolerance.disabled = not enable_dimensional.value
    
    filter_volume_check.observe(update_widgets, names='value')
    enable_flow.observe(update_widgets, names='value')
    enable_thermal.observe(update_widgets, names='value')
    enable_energy.observe(update_widgets, names='value')
    enable_dimensional.observe(update_widgets, names='value')
    
    print("✅ Callback functions attached!")


✅ Callback functions attached!


In [5]:
if WIDGETS_AVAILABLE:
    
    # Create loading panel
    loading_panel = widgets.VBox([
        widgets.HTML("<h2>📂 Load Volume</h2>"),
        HBox([
            file_path_text,
            file_format_dropdown
        ]),
        HBox([
            widgets.HTML("<b>Voxel Size:</b>"),
            voxel_size_x,
            voxel_size_y,
            voxel_size_z
        ]),
        sample_name_text,
        HBox([load_button, volume_info_display])
    ])
    
    # Create configuration panel
    config_panel = widgets.VBox([
        widgets.HTML("<h3>📋 Pipeline Configuration</h3>"),
        widgets.HTML("<b>Analysis Modules:</b>"),
        enable_segmentation,
        enable_preprocessing,
        enable_core_analysis,
        enable_porosity,
        enable_filament,
        enable_flow,
        enable_thermal,
        enable_energy,
        enable_dimensional,
        enable_uncertainty,
        widgets.HTML("<hr>"),
        widgets.HTML("<b>Parameters:</b>"),
        segmentation_method,
        HBox([filter_volume_check, min_volume]),
        flow_direction,
        thermal_conductivity,
        temperature_gradient,
        HBox([design_dim_x, design_dim_y, design_dim_z]),
        tolerance
    ])
    
    # Create pipeline panel
    pipeline_panel = widgets.VBox([
        widgets.HTML("<h3>🚀 Pipeline Execution</h3>"),
        run_pipeline_button,
        pipeline_results_display,
        pipeline_visualization
    ])
    
    # Create export panel
    export_panel = widgets.VBox([
        widgets.HTML("<h3>💾 Export and Reporting</h3>"),
        output_directory,
        export_format,
        generate_report_check,
        export_dragonfly_check,
        create_reproducibility_check,
        export_button,
        export_status_display
    ])
    
    # Create tabs for organized display
    main_tabs = Tab(children=[
        config_panel,
        pipeline_panel,
        export_panel
    ])
    main_tabs.set_title(0, '📋 Config')
    main_tabs.set_title(1, '🚀 Pipeline')
    main_tabs.set_title(2, '💾 Export')
    
    # Create main dashboard
    dashboard = widgets.VBox([
        widgets.HTML("<h1>🚀 Complete Analysis Pipeline</h1>"),
        loading_panel,
        widgets.HTML("<hr>"),
        main_tabs,
        widgets.HTML("<hr>"),
        progress_bar,
        status_display
    ])
    
    # Display the dashboard
    display(dashboard)
    print("\n✅ Dashboard displayed! Configure and run your complete analysis pipeline.")
    print("\n💡 Tips:")
    print("   1. Load your XCT volume")
    print("   2. Select which analysis modules to include")
    print("   3. Configure parameters for each module")
    print("   4. Run the complete pipeline")
    print("   5. Export results in multiple formats")
    print("   6. Generate reports and reproducibility packages")
    
else:
    print("❌ Cannot display dashboard - ipywidgets not available")



✅ Dashboard displayed! Configure and run your complete analysis pipeline.

💡 Tips:
   1. Load your XCT volume
   2. Select which analysis modules to include
   3. Configure parameters for each module
   4. Run the complete pipeline
   5. Export results in multiple formats
   6. Generate reports and reproducibility packages


## 6. Summary

### What We Learned

1. **Complete Pipeline Workflow**:
   - End-to-end analysis from data loading to reporting
   - Integration of all analysis modules
   - Configurable analysis steps
   - Progress tracking through all steps

2. **Multi-Format Export**:
   - JSON export for programmatic access
   - CSV export for spreadsheet analysis
   - Excel export with multiple sheets
   - HTML report generation

3. **Reproducibility**:
   - Reproducibility package creation
   - Configuration saving
   - Provenance tracking
   - Seed management

4. **Integration**:
   - DragonFly software export
   - External tool compatibility
   - Workflow sharing

### Key Insights

- **Complete pipeline** enables efficient end-to-end analysis
- **Modular design** allows selective analysis
- **Multiple export formats** support different use cases
- **Reproducibility packages** ensure analysis can be repeated
- **Integration capabilities** enable collaboration with other tools

### Pipeline Workflow

```
Load Data → Segment → Preprocess → Core Analysis → 
Porosity → Filament → Flow → Thermal → Energy → 
Dimensional → Uncertainty → Export → Report
```

### Best Practices

1. **Start with core analysis** to understand basic structure
2. **Add experimental analysis** for performance insights
3. **Include quality control** for validation
4. **Export in multiple formats** for different audiences
5. **Create reproducibility packages** for sharing

### Resources

- [Framework Documentation](../docs/README.md)
- [All Modules](../docs/modules.md)
- [Workflows](../docs/workflows.md)
- [Tutorials](../docs/tutorials.md)

---

## 🎉 Congratulations!

You've completed all 8 notebooks! You now have a comprehensive understanding of:

- ✅ Basic XCT data analysis
- ✅ Preprocessing and data cleaning
- ✅ Core morphological analysis
- ✅ Experimental analysis (flow, thermal, energy)
- ✅ Advanced analysis (sensitivity, virtual experiments)
- ✅ Comparative analysis and batch processing
- ✅ Quality control and validation
- ✅ Complete end-to-end pipeline

**Next Steps:**
- Apply these techniques to your own data
- Customize workflows for your research needs
- Contribute improvements to the framework
- Share your results and insights!
